In [4]:
import sys
sys.dont_write_bytecode = True
import numpy as np
from testing_suite import *
from parameters import *
from algorithm import *
import functions as fcn
import time

In [ ]:
c_fp = np.array([
    [172 - 137*43**0.5], 
    [215 + 44*43**0.5], 
    [49*(1003*43**0.5 - 989)/(288*math.pi)]
]) # N+1 x N-M coefficients c_fp[n',mu] [!] this line is specifically 2D

theory_parameters = Parameters(code='T', beta_functions=None, fixed_point=None, true_result=0.15, linear_generating_function=None, M=1)
meta_parameters = Parameters(code='M', collocation_point_numbers=None, collocation_point_bounds=None, basis_functions=None, basis_functions_derivatives=None, fix_parameters=None, population_size=None, algorithm=None)
control_parameters = Parameters(code='C', fitness_function=None, selection=None, mutation=None, crossover=None, iteration_cap=5000)

In [ ]:
def add(a,b):
    return a+b

def mul(a,b):
    return a*b

test_parameters = Parameters(code='Z', small=np.arange(1,11), medium=np.arange(0,120,20), large=np.arange(0,1250,250), result=0.15, functions=np.array([add, mul]))
print(test_parameters.get_option_counts())
print(test_parameters.get_combination_number())

test_parameters.code


In [5]:
for x in range(10):
    print(x, end='\r')
    time.sleep(1)
print()

9
